This notebook creates a baseline model for predicting dementia from linguistic data.

Data source: Ram Balasubramanium at Zelar Health.  
Confirm: data originally from https://dementia.talkbank.org/access/English/Pitt.html

Python library for parsing chat files:
https://pylangacq.org/

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from copy import deepcopy
import re
import pylangacq

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

## Load Data
Extract data from chat transcript files into pandas DataFrame

In [ ]:
files_control = 'PittData/PittTranscripts/Control/cookie/'
files_dementia = 'PittData/PittTranscripts/Dementia/cookie/'

In [ ]:
# Count files in directories
file_count_control = !ls $files_control | wc -l
file_count_dementia = !ls $files_dementia| wc -l
all_files_count = int(file_count_control[0]) + int(file_count_dementia[0])
print('Control files:', file_count_control[0])
print('Dementia files:', file_count_dementia[0])
print('All files:', all_files_count)

In [ ]:
# Load data into chat reader
raw_data = pylangacq.Reader.from_dir(files_control)
raw_data.append(pylangacq.Reader.from_dir(files_dementia))

In [ ]:
# Create index from filenames and check index structure
idx = [i['Media'].split(',')[0] for i in raw_data.headers()]
print('Index length:', len(idx))
print('Are all index values unique?', len(idx) == len(set(idx)))

In [ ]:
# Set column names for features and output variables
cols = ['Group', 'MMSE', 'INV_Interjections', 'Repeats']

#### Extract output variables
* _Control/Dementia_: Set binary variable for control to 0 and any dementia diagnosis to 1
* _MMSE_: Not all files have a recorded MMSE value

In [ ]:
# Examine group values
values = [i['Participants']['PAR']['group'] for i in raw_data.headers()]
print('Unique group values:\n', set(values))

In [ ]:
print("Number of occurences of blank group value:", values.count(''))

Solution: If blank group value is assigned to Control, binary variable assignment matches original file designation.

Optional further exploration: track down the file with the blank control value to confirm it was corrected assigned to the control group OR delete it from the raw dataset before extracting other values

In [ ]:
# Set binary variable for Control=0 and Any Dementia Diagnosis=1
group = [0 if i['Participants']['PAR']['group'] == 'Control' or i['Participants']['PAR']['group'] == ''
         else 1
         for i in raw_data.headers()]
print('Dementia datapoints:', np.array(group).sum())

In [ ]:
# MMSE was coded by transcribers in the 'education' key in each transcript's header
MMSE = [int(i['Participants']['PAR']['education']) if i['Participants']['PAR']['education'] != ''
        else None
        for i in raw_data.headers()]

#### Extract feature variables

In [ ]:
# Extract number of interjections by investigator per interview
INV = [len(i) for i in raw_data.utterances(participants="INV", by_files=True)]

In [ ]:
# Extract number of word or phrase repetitions
repeat_word = '\[/\]'
repeat_phrase = '\[//\]'

reps = []
for file in raw_data.utterances(by_files=True):
    # Collect each file's utterances into a single string to search using regular expressions
    utts_list = []
    for utterance in file:
        utts_list.append(utterance.tiers.get('PAR', ''))
    utts = "".join(utts_list)
    # add each file's number of repeated words and phrases to the variable
    reps.append(len(re.findall(repeat_word, utts)) + \
                len(re.findall(repeat_phrase, utts)))    

In [ ]:
# Load into DataFrame
data = pd.DataFrame(list(zip(group, 
                             MMSE,
                             INV,
                             reps
                             )),
                    columns=cols, 
                    index=idx)

In [ ]:
# Check combined index structure
print('Index length:', data.index.size)
print('Are all index values unique?', len(data.index) == len(set(data.index)))

In [ ]:
# Name index and dataframe
data.index.name = 'Filename'
data.name = 'Dementia Study - Cookie Theft Data'

## Explore Data

In [ ]:
print("Rows:", data.shape[0], "Columns:", data.shape[1])
data.head(5)

In [ ]:
print('Index data type:', data.index.dtype)
data.dtypes
# Note: MMSE is interpreted by Python as type float because of missing MMSE data - NaN is type float

In [ ]:
data.info()

In [ ]:
# Control group Summary statistics
data[data.Group==0].describe()

In [ ]:
# Dementia group Summary statistics
data[data.Group==1].describe()

In [ ]:
# Review missing values
print('{0:<20} {1:^30}'.format('Samples Total', data.shape[0]))
print()
print('{0:<20} {1:^30}'.format('Variable', 'Samples with Missing Data'))
print()
# Data Index
print('{0:<20} {1:^30}'.format(data.index.name, pd.isna(data.index).sum()))
# Each column
for i in range(len(cols)):
    print('{0:<20} {1:^30}'.format(cols[i], pd.isna(data[cols[i]]).sum()))

Are variables normally distributed?  **No**

In [ ]:
fig = plt.figure(figsize = (16,4)) 
for i in range(len(cols)):
    ax = fig.add_subplot(1, 4, i+1)
    ax.hist(data[cols[i]].dropna(), color='mediumvioletred') 
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["left"].set_visible(False)
    ax.yaxis.set_ticks_position('none')
    ax.set_yticklabels('')
    ax.set_xlabel(cols[i])

## Baseline models
For comparison, baseline models created from the dataset used for ADReSS challenge for the 2020 Interspeech Conference reached 75% accuracy and f1 scores using a larger subset of linguistic data from the same raw dataset.\*

The dataset used for the ADreSS challenge was an age and gender-matched subset of the full Pitt dataset, and included spontaenous speech. The model used 34 linguistic features extracted from the raw dataset, including duration, total utterances, MLU (mean length of utterance), type-token ratio, open-closed class word ratio, and percentages of 9 parts of speech. 

The baseline model created in this notebook uses the portion of the Pitt dataset in which participants are asked to describe the cookie theft picture commonly used in aphasia testing and uses only two features: number of interjections by interviewer and number of repeated words and phrases by the participant.


\* Luz S, Haider F, de la Fuente S, Fromm D, MacWhinney B. August 2020. *Alzheimer’s Dementia Recognition through Spontaneous Speech: The ADReSS Challenge.* https://arxiv.org/abs/2004.06833v3  

In [ ]:
# Shuffle data
data = data.sample(frac=1, random_state=8)
print("Rows:", data.shape[0], "Columns:", data.shape[1])

### Model 1:  Predict Dementia vs. Control Group
* Output variable is binary: Control vs. Dementia Group
* Model types: Logistic Regression and Random Forest
* Training to Test set ratio:  70%/30%
* Variations of input variable: 
    - Single input feature:  Number of interjections by the Investigator.
    - Single input feature:  Number of repeated words and phrases by participant
    - Both features combined
    

Number of interviewer interjections is a potentially biased variable, if interviewer had knowledge of participant's dementia status or MMSE score before the interview.  

TO DO:
Set up for cross-validation or bootstrapping for better model assessment. 

In [ ]:
def split_data(X, y):
    """Input: Input features (as data.SingleColumnName or data[['ColumnName', 'ColumnName']])
              Single output variable as data.SingleColumnName
              
       Output: Training and test set with consisitent size, stratification and random_state
    """
    X_train, X_test, y_train, y_test = train_test_split(X,
                                                        y, 
                                                        test_size = 0.3,
                                                        stratify = y, 
                                                        random_state = 32)
    return X_train, X_test, y_train, y_test

First iteration of this model uses only one feature: number of interjections by interviewer

In [ ]:
X_train, X_test, y_train, y_test = split_data(data.INV_Interjections, data.Group)

In [ ]:
# confirm equal control vs. dementia split in train vs. test sets
print('Full group % dementia:', round(data['Group'].mean(), 4))
print('Training set % dementia:', round(y_train.mean(), 4))
print('Test set % dementia:', round(y_test.mean(), 4))

In [ ]:
logm = LogisticRegression()
log_baseline = logm.fit(pd.DataFrame(X_train), y_train)

In [ ]:
# Assess fit of model
print('Accuracy of baseline model is:')
print(round(log_baseline.score(pd.DataFrame(X_test), y_test), 2))
# AOC was similar to f1_score in most cases for this data
#print('Area under the ROC curve is:')
#print(round(roc_auc_score(y_test, log_baseline.predict_proba(pd.DataFrame(X_test))[:, 1]), 2))
print('F1 score is:')
print(round(f1_score(y_test, log_baseline.predict(pd.DataFrame(X_test))), 2))

Same model with different input feature: Number of repeated words and phrases per interview.

In [ ]:
X_trainB, X_testB, y_trainB, y_testB = split_data(data.Repeats, data.Group)

In [ ]:
log_baselineB = logm.fit(pd.DataFrame(X_trainB), y_trainB)

In [ ]:
# Assess fit of model
print('Accuracy of 2nd baseline model is:')
print(round(log_baselineB.score(pd.DataFrame(X_testB), y_testB), 2))
print('F1 score is:')
print(round(f1_score(y_testB, log_baselineB.predict(pd.DataFrame(X_testB))), 2))

Same model using both features

In [ ]:
X_trainC, X_testC, y_trainC, y_testC = split_data(data[['Repeats', 'INV_Interjections']], data.Group)

In [ ]:
log_baselineC = logm.fit(pd.DataFrame(X_trainC), y_trainC)

In [ ]:
# Assess fit of model
print('Accuracy of 3rd baseline model is:')
print(round(log_baselineC.score(pd.DataFrame(X_testC), y_testC), 2))
print('F1 score is:')
print(round(f1_score(y_testC, log_baselineC.predict(pd.DataFrame(X_testC))), 2))

Try random forest model with same data split

In [ ]:
rf = RandomForestClassifier()
rf_baseline = rf.fit(pd.DataFrame(X_trainC), y_trainC)

In [ ]:
# Assess fit of model
print('Accuracy of random forest baseline model, without any hyperparameter tuning, is:')
print(round(rf_baseline.score(pd.DataFrame(X_testC), y_testC), 2))
print('F1 score is:')
print(round(f1_score(y_testC, rf_baseline.predict(pd.DataFrame(X_testC))), 2))

### Model 2: Prediction of MMSE

Same parameters as Model 1 execpt:
* Output variable: MMSE score ranges, binned according to density of MMSE value ranges in the dataset

In [ ]:
# first remove datapoints without MMSE scores
data_MMSE = data[data.MMSE.notna()]
print("Rows:", data_MMSE.shape[0], "Columns:", data_MMSE.shape[1])

In [ ]:
# Bin MMSE scores to use in a classification model
bins = KBinsDiscretizer(n_bins=4, strategy="quantile", encode="ordinal")
bins.fit(pd.DataFrame(data_MMSE.MMSE))
data_MMSE['MMSE_bins'] = bins.transform(pd.DataFrame(data_MMSE.MMSE))

In [ ]:
data_MMSE.describe()

Bin edges are set by default by KBinsDiscretizer based on frequency of values.  Bin edges set based on diagnostic criteria would be more meaningful.

In [ ]:
bins.bin_edges_

In [ ]:
X_train2, X_test2, y_train2, y_test2 = split_data(data_MMSE[['Repeats', 'INV_Interjections']], 
                                                  data_MMSE.MMSE_bins)

In [ ]:
log_baselineMMSE = logm.fit(pd.DataFrame(X_train2), y_train2)

In [ ]:
# Assess fit of model
print('Accuracy of baseline model predicting MMSE with logistic regression is:')
print(round(log_baselineMMSE.score(pd.DataFrame(X_test2), y_test2), 2))
print('F1 score is:')
print(round(f1_score(y_test2, log_baselineMMSE.predict(pd.DataFrame(X_test2)), average='weighted'), 2))

In [ ]:
rf_baselineMMSE = rf.fit(pd.DataFrame(X_train2), y_train2)

In [ ]:
print('Accuracy of baseline model predicting MMSE with random forest classifier is:')
print(round(rf_baselineMMSE.score(pd.DataFrame(X_test2), y_test2), 2))
print('F1 score is:')
print(round(f1_score(y_test2, rf_baselineMMSE.predict(pd.DataFrame(X_test2)), average='weighted'), 2))

The MMSE baseline model obviously isn't working well and needs work.  Next steps:  
* track down f1 scores per bin.  Predictions may be better for some MMSE scores than others. 
* Could set up manual bins for MMSE based on diagnostic criteria.  For instance:  1-12, 13-20, 21-24, 25-27, 28-30.
* MMSE prediction might also require more features, or some hyperparameter tuning